# Predicting ASD diagnosis from Genetic Data

0/1 classification with logistic regression is a well-studied problem.  In order to familiarize myself with standard logistic regression techniques, I'm going to start with the simple two-class classification problem of predicting ASD/non-ASD diagnosis from genotype.

Author: Rachael Caelie "Rocky" Aikens

Created: Oct 25, 2017

Version: 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn import linear_model
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import f1_score, make_scorer, roc_curve
import random

## Data preprocessing

We have genotype information for siblings from the Agre and Simons Simplex Collection, which has been featurized into a binary matrix (described below). In addition to that, we have imputed ASD/non-ASD labels and ADOS/ADI-R scores for a subset of those individuals.

### Feature Data (Genotype)

The input data is represented as a binary matrix.  There are a couple different representations we can use here, but to start I will use 1 = loss of function variant(compound het or homozygous alternate), 0 = no loss of function variant.

In [ ]:
# load input feature dataset for Agre
Agre_asd = pd.read_csv("../../../iHART/kpaskov/CGT/data/v34_lof_asd_af0.50.txt", index_col=0).transpose()
Agre_ctrl = pd.read_csv("../../../iHART/kpaskov/CGT/data/v34_lof_typical_af0.50.txt", index_col=0).transpose()

print "Cases: ", Agre_asd.shape[0]
print "Controls: ", Agre_ctrl.shape[0]

In [ ]:
# load input feature dataset for SSC
SSC_asd = pd.read_csv("../../../iHART/kpaskov/CGT/data/SSC_lof_asd_af0.50.txt", index_col=0).transpose()
SSC_ctrl = pd.read_csv("../../../iHART/kpaskov/CGT/data/SSC_lof_typical_af0.50.txt", index_col=0).transpose()

print "Cases: ", SSC_asd.shape[0]
print "Controls: ", SSC_ctrl.shape[0]

In [ ]:
# merge SSC and Agre data
X_asd = pd.concat([SSC_asd, Agre_asd], axis = 0).fillna(0)
X_ctrl = pd.concat([SSC_ctrl, Agre_ctrl], axis = 0).fillna(0)

In [ ]:
X = pd.concat([X_asd, X_ctrl], axis=0)
print "Total cases: ", X_asd.shape[0]
print "Total controls: ", X_ctrl.shape[0]
print "Features (ie. genes): ", X.shape[1]
print "Missing Values: ", int(X.isnull().values.any())

### Target Data (ASD/non-ASD diagnosis)

We have a file that Kelley has made with inferred Autism/Control diagnosis for the individuals in the iHart study.  We will try and predict diagnosis 0 = Control, 1 = Austism.

In [ ]:
y = pd.read_csv("../../../iHART/kpaskov/PhenotypeGLRM/data/all_samples_filtered_labels.csv", usecols = ['identifier','diagnosis'], index_col=0)

In [ ]:
# shift y to a 0/1 representation for Control/ASD
y["diagnosis"] = np.where(y['diagnosis'] == 'Autism', 1, 0)

### Filtering for Overlap

Our phenotype labels y may not perfectly overlap with our genotype data, X.

In [ ]:
# get lists of individuals in X and Y
m_x = X.index.values.tolist()
m_x_asd = X_asd.index.tolist()
m_x_ctrl = X_ctrl.index.tolist()
m_y = y.index.values.tolist()

# check subject overlap between X and Y
print "%d subjects in X are not in y.  Of these, %d are cases and %d are controls." % (len(set(m_x) - set(m_y)), len(set(m_x_asd) - set(m_y)), len(set(m_x_ctrl) - set(m_y)))

# make a list of Subject IDs with overlap
subjects = list(set(m_x) & set(m_y))
print "This leaves %d subjects: %d cases and %d controls." % (len(subjects), len(set(m_x_asd) & set(m_y)), len(set(m_x_ctrl)&set(m_y))) 

**Note:** The set of "cases" and "controls" appear to be differently defined between the iHart Phenotype labels (i.e. our `y` labels) and the CGT matrix labels (i.e. our `X` features). 

You can notice that the majority of controls don't appear in our phenotype information dataset. This is because ADOS\ADI-R was not administered to many controls from SSC and Agre. Since we're interested in classifying ASD/non-ASD, for our purposes it is not necessary to exclude these individuals because we do not necessarily need any phenotype information outside of diagnosis. Rather, we can infer that all individuals in a 'control' CGT matrix without ADOS/ADI-R information have a non-ASD diagnosis.

In [ ]:
to_add = list(set(m_x_ctrl) - set(m_y))
y_ctrl = pd.DataFrame(np.zeros(len(to_add),), columns = ['diagnosis'],index = to_add)
y = pd.concat([y, y_ctrl], axis = 0)
subjects = subjects + to_add
print len(subjects)
print y.shape

In [ ]:
# redefine X and Y to contain only the subjects we want
X = X.ix[subjects]
y = y.ix[subjects]

# check we have the same subject IDs in the same order for X and Y
print y.index.values.tolist() == X.index.values.tolist()
y = y.ix[:,0]
print y.value_counts()

One thing that's probably going to be an issue for this experiment is that there are very few controls for whom we have both genetic and ADOS/ADI-R information.  This is going to mean that a random classifier performs with fairly high accuracy, just because classifying most or all individuals as autistic is a effective strategy when we have so few negatives. 

## Data Splitting

Since we have ~1,600 examples, I'm going to hold out 20% of the data as a test set and then do 5 fold cross validation using built-in sklearn methods.

In [ ]:
random.seed(143)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## Logistic Regression

### Model and training parameters

I am going to implement logistic regression using sklearn.

We'll start with the following parameters:

**Cost function**
- Penalty distance metric = $L_2$
- Dual formulation = `False` (better when $m$ > $n$)
- c ($\frac{1}{\lambda}$ for regularization) = 1

**Optimization Algorithm**
- tolerance for convergence = $1\times 10^{-4}$
- optimization algorithm = liblinear

**Model definition**
- fit_intercept = `True`
- class weighting = None
- multi_class = 'ovr'

More or less, these are the sklearn defaults, which I can tune at a later point.

I've built a python object called EvalLR which will help me run cross validation for my regression models and output plots and statistics.  The following code initializes an EvalLR with the logistic regression model described above:

In [ ]:
# Import EvalLR
import class_EvalLR
reload(class_EvalLR)
from class_EvalLR import EvalLR

### First-pass 5-fold cross validation

To start, I'm going to run 5-fold cross validation on the training set. The following code does the necessary split for the data and prints the train and test scores for each fold using the f1 scoring metric.  Recall that this is:

$$F_1 = \frac{2}{\frac{1}{r} + \frac{1}{p}} = \frac{2rp}{r + p},$$

where $r$ represents the *recall* or *sensitivity* of the classification and $p$ represents the *precision*. 

In [ ]:
evalr = EvalLR(X_train, y_train, reg = 'l2')

In [ ]:
scores, topvals = evalr.kfold(7, True)

Below are the training and testing F1 scores for each fold of cross validation:

In [ ]:
print scores
print "Train:", np.mean(scores.Train_score)
print "Test:", np.mean(scores.Test_score)

In addition to training a predictive model, we'd also like to be able to make inference about which of the features in our CGT matrix are most informative for predicting Austism status.  The following are the top 10 genes with the greatest average odds ratios (i.e. farthest from 1). In order to make statistical inference, however, we should calculate a Wald statistic for each feature and search for the most significantly predictive features using multiple hypothesis testing correction.

In [ ]:
print topvals

## Regularization

Since our testing performance f1-scores are still about .1 below our training scores, it makes some sense to look into tuning our regularization parameter $C$ to avoid overfit.  I've written a function below, `reg_plot`, which performs 5 fold cross validation for models with different values of $C$.  

**Note** Recall that $C$ is the inverse of the cannonical regularization parameter, $\lambda$, so that smaller $C$ corresponds to stronger regularization.

In [ ]:
# create a plot of preformance versus f1 score for different c values
def reg_plot(c_vals, X_train, y_train, resample = False):
    c_scores = []
    print "Running 7-fold cross validation for:"
    for i in range(len(c_vals)):
        #print "C = %f" % c_vals[i]
        evalr = EvalLR(X_train, y_train, reg = 'l2', c = c_vals[i])
        c_scores.append(np.mean(evalr.kfold(7, False, False, resample)[0].Test_score))

    plt.clf()
    plt.ylabel('Feature 2')
    plt.xlabel('Feature 1')
    plt.plot(c_vals, c_scores, linestyle = '-')
    plt.show()
    plt.figure(figsize=(10,10))
    return c_vals[c_scores.index(max(c_scores))]

In [ ]:
c_vals = [2**5, 2**4, 2**3, 2**2, 2, 2**-1, 2**-2, 2**-3, 2**-4, 2**-5]
c_opt = reg_plot(c_vals, X_train, y_train)

At a first pass, it seems like lower regularization parameters may increase performance on the testing set.  Let's zoom in on performance for $C \leq 1$ (since 1 is the default value we've previously been using):

In [ ]:
c_vals = [1, 2**-1, 2**-2, 2**-3, 2**-4, 2**-5, 2**-6, 2**-7, 2**-8, 2**-9, 2**-10]
c_opt = reg_plot(c_vals, X_train, y_train)

These results suggest that stronger regularization may improve performance.  Let's try again with a stronger regularization (the max of the plot above).

In [ ]:
evalr = EvalLR(X_train, y_train, reg = 'l2', c = c_opt)
scores, topgenes = evalr.kfold(7, True)

In [ ]:
print scores
print "Train:", np.mean(scores.Train_score)
print "Test:", np.mean(scores.Test_score)

In [ ]:
print topgenes

## Resampling data to address class imbalance

Right now, we have a class imbalance problem because we tend to have many more autistic than non-autistic people in our dataset.  In general, we currently have two autistic subjects for every control subject in our training set:

In [ ]:
y_train.value_counts()

One approach to address this problem is to resample from our data so that there is a 1:1 raio between austistic subjects and non-autistic controls.  In this implementation, I oversample the neurotypical subjects; that is, the positive subjects in the training data remain the same, but I sample with replacement from the neurotypical subjects in the training set for each fold so that there are as many controls as ASD cases in both the training and testing sets.

Now we can retrain our model with resampled data:

In [ ]:
evalr = EvalLR(X_train, y_train, reg = 'l2')
scores, topgenes = evalr.kfold(7, makeROC = True, resample = True); print scores; print topgenes

After resampling, the ROC curves look about the same.  The average training score is increased, and the average testing error is about the same as before resampling with this regularization parameter. 

We can tune the regularization parameter as we did before:

In [ ]:
c_vals = [2**6, 2**5, 2**4, 2**3, 2**2, 2, 2**-1, 2**-2, 2**-3, 2**-4, 2**-5]
reg_plot(c_vals, X_train, y_train, resample = True)

In [ ]:
c_vals = [1, 2**-1, 2**-2, 2**-3, 2**-4, 2**-5, 2**-6, 2**-7, 2**-8, 2**-9, 2**-10]
c_opt = reg_plot(c_vals, X_train, y_train, resample = True)

In [ ]:
evalr = EvalLR(X_train, y_train, reg = 'l2', c = c_opt)
scores, topgenes = evalr.kfold(7, makeROC = True, resample = True); print scores; print topgenes

## Improvements

The following changes might be good to think about:
- continue to improve EvalLR for fine-tuning regression models.
- get wald statistic to detect important features
- try other regression methods
- is f1 score the best method when class imbalance is high?  Maybe I should be using AU-ROC or AU-PRC.
- tune regularization parameter to try and reduce the variance we have
- upsample to avoid class imbalance
- split train/test data based on family ID just like sibkfold
- add precision-recall